In [46]:
import pandas as pd
from sklearn.metrics import mean_squared_error as mse # rsme = mean_squared_error(y_true, y_pred, squared=False)
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV as rcv
import sklearn.preprocessing as pp
import numpy as np
from joblib import parallel_backend

In [47]:
#importing and reading the csv
directory = 'C:\\Users\\Emma Hegermiller\\Git\\price-prediction\\data'

df_train = pd.read_csv(directory + '\\train.csv').drop(columns=['Unnamed: 0'])
df_test = pd.read_csv(directory + '\\test.csv').drop(columns=['Unnamed: 0'])

In [48]:
X_train = df_train.drop(columns = ["list_price", "mileage", "accident_count", 
                                   "fuel_economy_city", "fuel_economy_highway", "msrp"])
X_test = df_test.drop(columns = ["list_price", "mileage", "accident_count", 
                                   "fuel_economy_city", "fuel_economy_highway", "msrp"])
y_train = df_train.list_price
y_test = df_test.list_price

test_missing = []
for col in X_train.columns:
    if col not in X_test.columns:
        test_missing.append(col)
    else:
        pass

train_missing = []
for col in X_test.columns:
    if col not in X_train.columns:
        train_missing.append(col)
    else:
        pass

test_add = pd.DataFrame(columns=test_missing)
X_test = X_test.join(test_add).fillna(0)

train_add = pd.DataFrame(columns=train_missing)
X_train = X_train.join(train_add).fillna(0)
   
pd.set_option('display.max_rows', 500)

print("Train columns: \n{}".format(X_train.dtypes))
print("Test missing: \n{}".format(len(test_missing)))
print("Train missing: \n{}".format(len(train_missing)))


Train columns: 
is_cpo                          int64
seller_is_franchise_dealer      int64
seller_is_online_only           int64
seller_ships_nationwide         int64
mileage_normalized            float64
                               ...   
seller_city_Williamsburg        int64
seller_city_Winchester          int64
seller_city_Wytheville          int64
seller_state_IA                 int64
seller_state_ME                 int64
Length: 690, dtype: object
Test missing: 
245
Train missing: 
205


In [49]:
# Ridge Regression Train
with parallel_backend('threading', n_jobs=8):
    ridge = rcv(cv=3).fit(X_train, y_train)
    ridge_params = ridge.get_params()
    r_squared = ridge.score(X_train, y_train)
    y_train_pred = ridge.predict(X_train)
    rmse_train = mse(y_train, y_train_pred, squared = False)

In [50]:
# Ridge Regression Test
with parallel_backend('threading', n_jobs=8):
    y_test_pred = ridge.predict(X_test)
    rmse_test = mse(y_test, y_test_pred, squared = False)

In [45]:
# Model results
print("Ridge Regression Parameters: {}".format(ridge_params))
print("Ridge Train R^2: {}".format(r_squared))
print("Ridge Train RMSE: {}".format(rmse_train))
print("Ridge Test RMSE: {}".format(rmse_test))

Ridge Regression Parameters: {'alpha_per_target': False, 'alphas': array([ 0.1,  1. , 10. ]), 'cv': 3, 'fit_intercept': True, 'gcv_mode': None, 'normalize': False, 'scoring': None, 'store_cv_values': False}
Ridge Train R^2: 0.9988431926361353
Ridge Train RMSE: 24.298963418192265
Ridge Test RMSE: 2001.6773053833676
